## Set-up

In [ ]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import gc
from matplotlib import colormaps as cm
from glob import glob

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import utils
import track

### Hyper parameters

In [ ]:
loadstar_weight_name = "loadstar_first"
magik_weight_name = "MAGIK_MP_MPN"

# Experiment parameters
dir_mode = True
input_path = r'/home/gideon/data/data_VE/test' # Change this to where you have the tiff file
output_path = r'/home/gideon/Documents/PhD/Projects/KAW/output' # Change this to where you want the output to be saved
pages_batch_size = 2
channel_width = 0.3 # Microns

# Loadstar parameters
alpha = 0.999
cutoff = 0.00000002

# MAGIK parameters
radius = 0.08 # Value between 0 and 1 where 1 is the full image hight
traj_min = 6
n_frames = 4

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

loadstar, magik = utils.load_models(loadstar_weight_name, magik_weight_name)

if dir_mode:
    image_paths = glob(input_path + '/*.tif')
else:
    image_paths = [input_path]

In [ ]:
for path in image_paths:
    print(path)

#### Test loadstar parameters

In [ ]:
utils.test_loadstar(image_paths[0], loadstar, alpha, cutoff, 1)

### Extract metadata

In [ ]:
tiffpage = tifffile.TiffFile(image_paths[1]).pages[0]
for tag in tiffpage.tags.values():
    print(tag.name, tag.value)    


In [ ]:
width = tiffpage.tags.get('ImageWidth').value # Pixels
height = tiffpage.tags.get('ImageLength').value # Pixels
x_resolution = tiffpage.tags.get('XResolution').value # Pixels per micron
y_resolution = tiffpage.tags.get('YResolution').value # Pixels per micron

# Very bad solution but it works temporarily
fintervals = []
for path in image_paths:
    tiffpage = tifffile.TiffFile(path).pages[0]
    ImageDescription = tiffpage.tags.get('ImageDescription').value.split('\n') # Seconds
    finterval = float(ImageDescription[4].split('=')[1])
    fintervals.append(finterval)

### Calculated paramters

In [ ]:
# Calculated parameters
x_resolution = x_resolution[0] / 1000000 # Pixels per micron
y_resolution = y_resolution[0] / 1000000 # Pixels per micron

pixel_size_x = 1/x_resolution # Microns per pixel
pixel_size_y = 1/y_resolution # Microns per pixel

channel_width_pixels = channel_width * x_resolution # Pixels
channel_width_norm = channel_width_pixels / width

radius_ratio = radius / channel_width_norm

print(f"Channel width: {channel_width_pixels} pixels")
print(f"Radius ratio: {radius_ratio} ratio")

## Detect

In [ ]:
import importlib
importlib.reload(utils)
detections  = utils.detect(image_paths, loadstar, pages_batch_size, alpha, cutoff)
utils.save_detections(detections, f"{output_path}/detections.csv", full=True)

## Track

In [ ]:
import importlib
importlib.reload(track)

detections = utils.load_detections(f"{output_path}/detections.csv")

detections = utils.warp_axis(detections, radius_ratio, axis_name='centroid-0')
tracks = track.predict_trajectories(magik, detections, radius, n_frames, traj_min)
tracks = utils.unwarp_axis(tracks, radius_ratio, axis_name='x')

# Convert to pixel values
tracks['x'] *= width
tracks['y'] *= height
track.save_trajectories(tracks, f"{output_path}/tracks.csv")

### Plot track

In [ ]:
tracks.head(-1)

In [ ]:
import importlib
importlib.reload(track)
track.plot_trajectories(tracks, image_paths, output_path, pages_batch_size)

## Further analyze the tracks

### Change x and y units

#### Add axis for lengths in microns

In [ ]:
tracks = track.load_trajectories(f"{output_path}/tracks.csv")

tracks['y_microns'] = tracks['y'] * pixel_size_y
tracks['x_microns'] = tracks['x'] * pixel_size_x

#### Alt. replace x and y with micron versions

In [ ]:
tracks = track.load_trajectories(f"{output_path}/tracks.csv")

tracks['y'] = tracks['y'] * pixel_size_y
tracks['x'] = tracks['x'] * pixel_size_x

### Remove still particles

In [ ]:
still_cuttoff = 10

print(f'Number of objects before: {len(tracks[tracks["set"] == 0].entity.unique())}')

tracks_stills_removed = track.remove_still_objects(tracks, still_cuttoff)

print(f'Number of objects after: {len(tracks_stills_removed[tracks_stills_removed["set"] == 0].entity.unique())}')

In [ ]:
tracks_stills_removed.head(-1)

### Combine tracks

In [ ]:
track.plot_trajectories(tracks_stills_removed, image_paths, output_path, pages_batch_size, stop=50)

### Track length and distance

#### Distance Traveled

In [ ]:
import importlib
importlib.reload(track)

tracks = tracks_stills_removed

tracks = track.distance_traveled(tracks)

tracks = tracks.sort_values(by=['frame', 'entity', 'set'], ascending=True)
tracks.head(-1)

#### Appearances & time

In [ ]:
tracks = track.count_appearances(tracks)

tracks.head(-1)

In [ ]:
for set_num in tracks["set"].unique():
    tracks.loc[
            tracks["set"] == set_num, ["t"]
        ] = tracks[tracks['set'] == set_num]['frame_count'] * fintervals[set_num]

track.save_trajectories(tracks, f"{output_path}/tracks.csv")

### Extract intensity

In [ ]:
tracks = utils.detection_intensity(image_paths, tracks, pages_batch_size, mode='pixel', kernel_size=3)
track_intensities = utils.track_intensity(tracks, mode='mean')

track.save_trajectories(tracks, f"{output_path}/tracks.csv")

In [ ]:
tracks.head(-1)